In [4]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

import warnings
warnings.filterwarnings('ignore')

# Transfer Learning  Finetune FC layer

In [8]:
model_name = "resnet"
num_classes = 10
feature_extract = True

model_transfer, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/gm3044/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

0it [00:00, ?it/s]

Extracting ./datasets/CIFAR-10/cifar-10-python.tar.gz to ./datasets/CIFAR-10
Files already downloaded and verified


In [10]:
model_transfer = model_transfer.to(device)
params_to_update = model_transfer.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_transfer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [13]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_transfer.pth'
device = "cuda"
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("Standard_Resnet_Transfer")

In [14]:
train(model_transfer, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_transfer, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 1.6231169136588837	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.007857088801227	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 1.2811612665195904	 lr: 0.001
Epoch [2/100]	 Validation Loss: 0.8829580153091044	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 1.217041926768125	 lr: 0.001
Epoch [3/100]	 Validation Loss: 0.8157951537566849	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 1.1810605672314345	 lr: 0.001
Epoch [4/100]	 Validation Loss: 0.7855270504951477	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 1.1646906643572366	 lr: 0.001
Epoch [5/100]	 Validation Loss: 0.7828300866899611	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 1.1466873434498488	 lr: 0.001
Epoch [6/100]	 Validation Loss: 0.7613090661507619	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.142103331320731	 lr: 0.001
Epoch [7/100]	 Validation Loss: 0.7426101678534399	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss:

Epoch [60/100]	 Training Loss: 1.069207685530338	 lr: 0.001
Epoch [60/100]	 Validation Loss: 0.6770626532880566	 lr: 0.001
Epoch [60/100]	
Epoch [61/100]	 Training Loss: 1.067069401521512	 lr: 0.001
Epoch [61/100]	 Validation Loss: 0.6637019605576238	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 1.0724254075218649	 lr: 0.001
Epoch [62/100]	 Validation Loss: 0.6738717473005946	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 1.0695636359322103	 lr: 0.001
Epoch [63/100]	 Validation Loss: 0.6769446602350548	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.0697336907277022	 lr: 0.001
Epoch [64/100]	 Validation Loss: 0.6769097949130626	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.0684480115275858	 lr: 0.001
Epoch [65/100]	 Validation Loss: 0.6765231014807013	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.0692082399602436	 lr: 0.001
Epoch [66/100]	 Validation Loss: 0.6804313606853727	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Traini

In [15]:
print("the clean accuracy is ")
test(val_loader, model_transfer)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 77 %


In [16]:
print("PGD")
pgd_attack_transfer = generate_adv(model_transfer, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_transfer, "cuda")

PGD
[Model loaded]
Acc: 0.00 %
adv model


In [17]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_transfer, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_transfer, "cuda")

fgsm
[Model loaded]
Acc: 18.76 %
adv model


In [18]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_transfer, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_transfer, "cuda")

nifgsm
[Model loaded]
Acc: 2.02 %
adv model


In [19]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_transfer, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_transfer, "cuda")

vmifgsm
[Model loaded]
Acc: 0.00 %
adv model


# Adversarial Training on PGD

In [20]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(model_transfer) 


In [21]:
params_to_update = net_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in net_adv.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

Params to learn:
	 fc.weight
	 fc.bias


In [23]:
train(net_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 5.637540827626768	 lr: 0.001
Epoch [1/100]	 Validation Loss: 2.4945161704775654	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.688544991681033	 lr: 0.001
Epoch [2/100]	 Validation Loss: 2.3034445847137066	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.331932678247047	 lr: 0.001
Epoch [3/100]	 Validation Loss: 2.3041754010357436	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.3292663572999217	 lr: 0.001
Epoch [4/100]	 Validation Loss: 2.3032776675646818	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.331259251250635	 lr: 0.001
Epoch [5/100]	 Validation Loss: 2.3027724103082585	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.3305080046739115	 lr: 0.001
Epoch [6/100]	 Validation Loss: 2.3011498571951177	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 2.328952040513763	 lr: 0.001
Epoch [7/100]	 Validation Loss: 2.3042136566548406	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 2

Epoch [61/100]	 Training Loss: 2.3310230003903283	 lr: 0.001
Epoch [61/100]	 Validation Loss: 2.301942203618303	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 2.329626933388088	 lr: 0.001
Epoch [62/100]	 Validation Loss: 2.307575446140917	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 2.3292656127754077	 lr: 0.001
Epoch [63/100]	 Validation Loss: 2.302678977386861	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 2.329783272560295	 lr: 0.001
Epoch [64/100]	 Validation Loss: 2.3014297364633296	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 2.3292187430974467	 lr: 0.001
Epoch [65/100]	 Validation Loss: 2.301723383649995	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 2.329780163362508	 lr: 0.001
Epoch [66/100]	 Validation Loss: 2.3043304515790335	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 2.3314465677646723	 lr: 0.001
Epoch [67/100]	 Validation Loss: 2.301847946794727	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training Los

In [24]:
print("the clean accuracy is ")
test(val_loader, net_adv)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 10 %


In [25]:
print("PGD")
pgd_attack_transfer = generate_adv(model_transfer, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_adv, "cuda")

PGD
[Model loaded]
Acc: 10.06 %
adv model


In [26]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_transfer, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_adv, "cuda")

fgsm
[Model loaded]
Acc: 10.07 %
adv model


In [27]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_transfer, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_adv, "cuda")

nifgsm
[Model loaded]
Acc: 10.02 %
adv model


In [28]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_transfer, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_adv, "cuda")

vmifgsm
[Model loaded]
Acc: 10.00 %
adv model
